In [2]:
import h5py
import numpy as np
import torch
from torch import nn
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random
# from tqdm.auto import tqdm
# from tqdm.notebook import tqdm
from SVWN3 import f_svwn3
from SVWN3_njit import f_svwn3_njit
import csv
import copy
import random
from NN_models import NN_2_256, NN_8_256, NN_8_64

/opt/software/python/envs/pytorch1_11/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def ref(x, y):
    ''' 
    returns reference energies for points of a reaction grid from Reference_data.csv
    '''
    hartree2kcal = 627.5095
    with open("Reference_data.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        k = 1
        if y == 391:
            k = hartree2kcal
        ref = []
        for n, i in enumerate(ref_file):
            if x <= n + 1 <= y:
                ref.append((i[0], float(i[2]) * k))

        return ref

def load_ref_energies():
    '''Returns {db_name: [equation, energy]}'''
    ref_e = { # Получение референсных энергий
        "MGAE109":ref(8, 116),
        "IP13":ref(155, 167),
        "EA13":ref(180, 192),
        "PA8":ref(195, 202),
        "DBH76":ref(251, 288) + ref(291, 328),
        "NCCE31":ref(331, 361),
        "ABDE4":ref(206, 209),
        # "AE17":ref(375, 391),
        "pTC13":ref(232, 234) + ref(237, 241) + ref(244, 248)
        } 
    return ref_e

def load_component_names():
    '''
    Returns {db_name: {id: {'Components': [...], 'Coefficients: [...]'
                                }
                            }
                        }
     which is a dictionary with Components and Coefficients data about all reactions
    '''
    with open("total_dataframe_sorted_final.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        ref = dict()
        current_database = None
        
        for n, line in enumerate(ref_file):
            line = np.array(line)
            if n == 0:
                components = np.array(line)
            else:
                reaction_id = int(line[0])
                reaction_database = line[1]
                reaction_component_num = np.nonzero(list(map(float, line[2:])))[0] + 2
                if reaction_database in ref:
                    ref[reaction_database][reaction_id] = {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}
                else: 
                    ref[reaction_database] = {reaction_id: {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}}
        return ref

In [4]:
def get_compounds_coefs_energy_v2(reactions, energies):
    '''Returns {id: 
                    {'Components': [...], 'Coefficients: [...]', 'Energy: float', Database: str
                                }
                            }
    which is a dictionaty from load_component_names with Energy information added
    '''
    data_final = dict()
    i = 0
    databases = load_ref_energies().keys()
    for database in databases:
        data = reactions[database]
        for reaction in data:
            data[reaction]['Energy'] = energies[database][reaction][1]
            data_final[i] = {'Database': database,
                         'Components': reactions[database][reaction]['Components'].astype(object),
                         'Coefficients': reactions[database][reaction]['Coefficients'].astype(np.float64),
                         'Energy': energies[database][reaction][1]
            
        }
            i += 1
        
    return data_final

In [5]:
def get_h5_names(reaction):
    '''reaction must be from the function get_compounds_coefs_energy_v2'''
    database_match = {
        'MGAE109': 'mgae109',
        'IP13': 'ip13',
        'EA13': 'ea13',
        'PA8': 'pa8',
        'DBH76': 'ntbh38',
        'NCCE31': 'ncce31',
        'ABDE4': 'abde4',
        'AE17': 'ae17',
        'pTC13': 'ptc13'
    }
    names = []
    for elem in reaction['Components']:
        database = database_match[reaction['Database']]
        names.append(f'{elem}.h5')
    return names

In [6]:
def add_reaction_info_from_h5(reaction):
    '''
    reaction must be from get_compounds_coefs_energy_v2
    returns merged descriptos array X, integration weights, 
    a and b densities and indexes for backsplitting
    
    Adds the following information to the reaction dict using h5 files from the dataset:
    Grid : np.array with grid descriptors
    Weights : list with integration weights of grid points
    Densities : np.array with alpha and beta densities data for grid points
    HF_energies : list of Total HF energy (T+V) which needs to be added to E_xc
    backsplit_ind: list of indexes where we concatenate molecules' grids
    '''
    X = np.array([])
    backsplit_ind = []
    HF_energies = np.array([])
    for component_filename in get_h5_names(reaction):
        with h5py.File(f'data/{component_filename}', "r") as f:
            HF_energies = np.append(HF_energies, f["ener"][:][0])
            X_raw = np.array(f["grid"][:])
            if len(X) == 0:
                X = X_raw[:, 3:-1]
            else:
                X = np.vstack((X, X_raw[:, 3:-1]))
            backsplit_ind.append(len(X))
    densities = X[:, 1:3]
    weights = X[:,0]
    X = X[:, 1:]

    labels = ['Grid', 'Weights', 'Densities', 'HF_energies', 'backsplit_ind']
    values = [X, weights, densities, HF_energies, backsplit_ind]
    for label, value in zip(labels, values):
        reaction[label] = torch.Tensor(value)

    return reaction

In [7]:
def make_reactions_dict():
    '''
    Returns a dict like {reaction_id: {*reaction info}} with all info available listed below:
    ['Database', 'Components', 'Coefficients', 'Energy', 'Grid', 'Weights', 'Densities', 'HF_energies', 'backsplit_ind']
    '''
    data = get_compounds_coefs_energy_v2(load_component_names(), load_ref_energies())
    for i in data.keys():
        data[i] = add_reaction_info_from_h5(data[i])

    return data
    

In [8]:
def get_local_energies(reaction, constants):
    calc_reaction_data = {}
    densities = reaction['Densities']
    local_energies = f_svwn3(densities, constants)
    calc_reaction_data['Local_energies'] = local_energies
    return calc_reaction_data


def add_calc_reaction_data(reaction, calc_reaction_data):
    calc_reaction_data['Weights'] = reaction['Weights']
    calc_reaction_data['Densities'] = reaction['Densities']
    return calc_reaction_data

def backsplit(reaction, calc_reaction_data):
    backsplit_ind = reaction['backsplit_ind'].type(torch.int)
    splitted_data = dict()
    stop = 0
    
    for i, component in enumerate(reaction['Components']):
        splitted_data[component] = dict()
        start = stop
        stop = backsplit_ind[i]
        for elem in ('Local_energies', 'Weights', 'Densities'):
            splitted_data[component][elem] = calc_reaction_data[elem][start:stop]
    return splitted_data


def integration(reaction, splitted_calc_reaction_data):
    molecule_energies = dict()
    for i, component in enumerate(reaction['Components']):
        molecule_energies[component] = torch.sum(splitted_calc_reaction_data[component]['Local_energies'] \
                                              * (splitted_calc_reaction_data[component]['Densities'][:,0] \
                                              + splitted_calc_reaction_data[component]['Densities'][:,1]) \
                                              * (splitted_calc_reaction_data[component]['Weights'])) \
                                              + reaction['HF_energies'][i]
    return molecule_energies


def get_energy_reaction(reaction, molecule_energies):
    hartree2kcal = 627.5095
    s = 0
    for coef, ener in zip(reaction['Coefficients'], molecule_energies.values()):
        s += coef*ener
    reaction_energy_kcal = s * hartree2kcal
    return reaction_energy_kcal


def calculate_reaction_energy(reaction, constants):
    local_energies = get_local_energies(reaction, constants)
    calc_reaction_data = add_calc_reaction_data(reaction, local_energies)

    splitted_calc_reaction_data = backsplit(reaction, calc_reaction_data)

    molecule_energies = integration(reaction, splitted_calc_reaction_data)
    
    reaction_energy_kcal = get_energy_reaction(reaction, molecule_energies)

    return reaction_energy_kcal

In [9]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [10]:
data = make_reactions_dict()

In [11]:
data[73] # take the easiest reaction H2 = 2H

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype=object),
 'Coefficients': array([ 2., -1.]),
 'Energy': 109.49,
 'Grid': tensor([[3.1106e-01, 0.0000e+00, 3.6975e-09,  ..., 0.0000e+00, 1.4859e-09,
          0.0000e+00],
         [3.1106e-01, 0.0000e+00, 3.1224e-07,  ..., 0.0000e+00, 1.2547e-07,
          0.0000e+00],
         [3.1106e-01, 0.0000e+00, 4.1824e-06,  ..., 0.0000e+00, 1.6807e-06,
          0.0000e+00],
         ...,
         [3.3523e-04, 3.3523e-04, 5.3741e-07,  ..., 5.3741e-07, 2.0039e-04,
          2.0039e-04],
         [4.5250e-04, 4.5250e-04, 9.8983e-07,  ..., 9.8983e-07, 2.7344e-04,
          2.7344e-04],
         [3.3523e-04, 3.3523e-04, 5.3741e-07,  ..., 5.3741e-07, 2.0039e-04,
          2.0039e-04]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.6055e-02, 2.2680e-02,
         2.6055e-02]),
 'Densities': tensor([[0.3111, 0.0000],
         [0.3111, 0.0000],
         [0.3111, 0.0000],
         ...,
         [0.0003, 0.0

In [12]:
def train_split(data, test_size, shuffle=False):
    if shuffle:
        keys = list(data.keys())
        random.shuffle(keys)
        for i in keys:
            data[keys[i]] = data[i]

    train, test = dict(), dict()
    border = round(len(data.keys()) * (1 - test_size))
    for i in range(len(data.keys())):
        if i <= border:
            train[i] = data[i]
        else:
            test[i] = data[i]
    return train, test


In [13]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        return self.data[i], self.data[i]['Energy']
    
    def __len__(self):
        return len(self.data.keys())

test_size = 0.2

data_train, data_test = train_split(copy.deepcopy(data), 0.2, True)


train_set = Dataset(data=data_train)
# train_dataloader = torch.utils.data.DataLoader(train_set, 
#                                                batch_size=1)

test_set = Dataset(data=data_test)
# test_dataloader = torch.utils.data.DataLoader(test_set, 
#                                               batch_size=1)

In [14]:
# device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')
device = torch.device('cpu')

model = NN_8_256().to(device)
model.load_state_dict(torch.load('predoptimized_2.param', map_location=device))

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))
criterion = nn.L1Loss()

In [15]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=20):
    train_loss_mae = []
    test_loss_mae = []


    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        model.train()
        # progress_bar = tqdm(train_loader)

        train_mae_losses_per_epoch = []
        for X_batch, y_batch in train_loader:
            X_batch_grid, y_batch = torch.Tensor(X_batch['Grid']).to(device), torch.Tensor(np.array([y_batch])).to(device)
            predictions = torch.expm1(model(X_batch_grid))
            reaction_energy = calculate_reaction_energy(X_batch, predictions)
            loss = criterion(reaction_energy, y_batch)
            print(X_batch['Components'], loss.item())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_mae_losses_per_epoch.append(loss.item())
        train_loss_mae.append(torch.mean(train_mae_losses_per_epoch))

        # #test
        # model.eval()
        # test_mae_losses_per_epoch = []
        # with torch.no_grad():
        #     for X_batch, y_batch in test_loader:
        #         X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
        #         preds = np.expm1(model(X_batch_grid))
        #         reaction_energy = calculate_reaction_energy(X_batch, predictions)
        #         loss = criterion(np.array(reaction_energy), np.array(y_batch))
        #         test_mae_losses_per_epoch.append(loss.item())
        # test_loss_mae.append(np.mean(test_mae_losses_per_epoch))
        # print(f'train MAE Loss = {train_loss_mae[epoch]:.8f}')
        # print(f'test MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae, preds[0].cpu().detach().numpy()

In [17]:
train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae, preds = train(model, criterion, optimizer, 
                                                                            train_set,
                                                                            test_set, n_epochs=20)

Epoch 1


/opt/software/python/jupyterhub2/lib/python3.10/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


['H2_htbh38' 'TS9_htbh38' 'F_nhtbh38'] nan
['PH2+_ip13' 'PH2_ip13'] nan
['H_mgae109' 'N_mgae109' 'NH2NH2_mgae109'] nan


KeyboardInterrupt: 

In [16]:
reaction = data[73]
reaction

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype=object),
 'Coefficients': array([ 2., -1.]),
 'Energy': 109.49,
 'Grid': tensor([[3.1106e-01, 0.0000e+00, 3.6975e-09,  ..., 0.0000e+00, 1.4859e-09,
          0.0000e+00],
         [3.1106e-01, 0.0000e+00, 3.1224e-07,  ..., 0.0000e+00, 1.2547e-07,
          0.0000e+00],
         [3.1106e-01, 0.0000e+00, 4.1824e-06,  ..., 0.0000e+00, 1.6807e-06,
          0.0000e+00],
         ...,
         [3.3523e-04, 3.3523e-04, 5.3741e-07,  ..., 5.3741e-07, 2.0039e-04,
          2.0039e-04],
         [4.5250e-04, 4.5250e-04, 9.8983e-07,  ..., 9.8983e-07, 2.7344e-04,
          2.7344e-04],
         [3.3523e-04, 3.3523e-04, 5.3741e-07,  ..., 5.3741e-07, 2.0039e-04,
          2.0039e-04]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.6055e-02, 2.2680e-02,
         2.6055e-02]),
 'Densities': tensor([[0.3111, 0.0000],
         [0.3111, 0.0000],
         [0.3111, 0.0000],
         ...,
         [0.0003, 0.0

In [17]:
X_batch = reaction
y_batch = reaction['Energy']
X_batch_grid, y_batch = X_batch['Grid'].to(device), torch.Tensor(np.array([y_batch])).to(device)

predictions = torch.expm1(model(X_batch_grid))

In [18]:
X_batch_grid.requires_grad, y_batch.requires_grad = True, True

In [19]:
predictions.shape, X_batch_grid.shape

(torch.Size([101502, 21]), torch.Size([101502, 7]))

In [20]:
predictions[:10].shape

torch.Size([10, 21])

In [21]:
true_constants = torch.Tensor([0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*torch.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1])
constants = torch.tile(true_constants, (101502,1))

In [32]:
torch.set_printoptions(precision=20)

In [33]:
constants[0,0]

tensor(0.03109069913625717163)

In [34]:
predictions[0]

tensor([ 3.11082098633050918579e-02,  1.55354645103216171265e-02,
         3.72747325897216796875e+00,  7.06047105789184570312e+00,
         1.29351100921630859375e+01,  1.80580902099609375000e+01,
        -1.04969792068004608154e-01, -3.25008869171142578125e-01,
         3.10632865875959396362e-02,  1.55571084469556808472e-02,
        -1.68711654841899871826e-02,  1.30721702575683593750e+01,
         2.01230430603027343750e+01,  1.06832981109619140625e+00,
         4.27197875976562500000e+01,  1.01578819274902343750e+02,
         1.14812364578247070312e+01, -4.09288644790649414062e-01,
        -7.43289113044738769531e-01, -2.28327989578247070312e-01,
         9.99994635581970214844e-01], grad_fn=<SelectBackward0>)

In [35]:
torch.abs((predictions[0]-constants[0])/constants[0]).max()*100

tensor(0.09296204149723052979, grad_fn=<MulBackward0>)

In [36]:
constants[0]

tensor([ 3.10906991362571716309e-02,  1.55453495681285858154e-02,
         3.72744011878967285156e+00,  7.06042003631591796875e+00,
         1.29351997375488281250e+01,  1.80578002929687500000e+01,
        -1.04979999363422393799e-01, -3.24999988079071044922e-01,
         3.10906991362571716309e-02,  1.55453495681285858154e-02,
        -1.68868638575077056885e-02,  1.30719995498657226562e+01,
         2.01231002807617187500e+01,  1.06834995746612548828e+00,
         4.27197990417480468750e+01,  1.01578002929687500000e+02,
         1.14813003540039062500e+01, -4.09285992383956909180e-01,
        -7.43294000625610351562e-01, -2.28343993425369262695e-01,
         1.00000000000000000000e+00])

In [43]:
from importlib import reload
import SVWN3
reload(SVWN3)

<module 'SVWN3' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/SVWN3.py'>

In [44]:
%%time
local_energies = get_local_energies(reaction, predictions) # 1k of 100k grid points for 2H = H2 system

1
1
1
1
1
1
1
1
CPU times: user 388 ms, sys: 24 ms, total: 412 ms
Wall time: 447 ms


In [45]:
local_energies['Local_energies'].isnan().sum()

tensor(258)

In [46]:
local_energies['Local_energies']

tensor([-0.66303867101669311523437500000000000000000000000000,
        -0.66305744647979736328125000000000000000000000000000,
        -0.66306197643280029296875000000000000000000000000000,
         ...,
        -0.08761207759380340576171875000000000000000000000000,
        -0.09582035988569259643554687500000000000000000000000,
        -0.08761709928512573242187500000000000000000000000000],
       grad_fn=<AddBackward0>)

In [47]:
tensor_sqrt = torch.load("tensor_sqrt.pt")

In [48]:
torch.sqrt(tensor_sqrt).isnan().any()

tensor(True)

In [49]:
part1 = torch.load("tensor_faux_part1.pt")
arc = torch.load("tensor_faux_arc.pt")
vwn = torch.load("tensor_faux_vwn.pt")
Q = torch.load("tensor_faux_Q.pt")
arc_arg = torch.load("tensor_faux_arc_arg.pt")
b = torch.load("tensor_faux_b.pt")
c = torch.load("tensor_faux_c.pt")

In [50]:
4*c[(arc*vwn).isnan()][0:10]

tensor([170.87919616699218750000000000000000000000000000000000,
        170.87927246093750000000000000000000000000000000000000,
        170.87927246093750000000000000000000000000000000000000,
        170.87672424316406250000000000000000000000000000000000,
        170.87797546386718750000000000000000000000000000000000,
        170.87751770019531250000000000000000000000000000000000,
        170.87734985351562500000000000000000000000000000000000,
        170.87797546386718750000000000000000000000000000000000,
        170.87751770019531250000000000000000000000000000000000,
        170.87689208984375000000000000000000000000000000000000],
       grad_fn=<MulBackward0>)

In [51]:
(b[(arc*vwn).isnan()]**2)[0:10]

tensor([170.87919616699218750000000000000000000000000000000000,
        170.87927246093750000000000000000000000000000000000000,
        170.87927246093750000000000000000000000000000000000000,
        170.87672424316406250000000000000000000000000000000000,
        170.87797546386718750000000000000000000000000000000000,
        170.87751770019531250000000000000000000000000000000000,
        170.87734985351562500000000000000000000000000000000000,
        170.87797546386718750000000000000000000000000000000000,
        170.87751770019531250000000000000000000000000000000000,
        170.87689208984375000000000000000000000000000000000000],
       grad_fn=<SliceBackward0>)

In [52]:
Q[(arc*vwn).isnan()]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [53]:
arc_arg[(arc*vwn).isnan()]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [54]:
part1.isnan().sum()

tensor(258)

In [55]:
arc.isnan().sum()

tensor(0)

In [56]:
vwn.isnan().sum()

tensor(0)

In [57]:
(arc*vwn)[(arc*vwn).isnan()]

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [58]:
arc[(arc*vwn).isnan()]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [59]:
vwn[(arc*vwn).isnan()]

tensor([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, i

In [61]:
part1

tensor([1.97768485546112060546875000000000000000000000000000,
        1.97768056392669677734375000000000000000000000000000,
        1.97768008708953857421875000000000000000000000000000,
         ...,
        1.61100685596466064453125000000000000000000000000000,
        1.63406682014465332031250000000000000000000000000000,
        1.61100447177886962890625000000000000000000000000000],
       requires_grad=True)

In [ ]:
calculate_reaction_energy(reaction, predictions)

In [92]:
def calculate_reaction_energy(reaction, constants):
    local_energies = get_local_energies(reaction, constants)
    print(calc_reaction_data)
    calc_reaction_data = add_calc_reaction_data(reaction, local_energies)
    print(calc_reaction_data)
    splitted_calc_reaction_data = backsplit(reaction, calc_reaction_data)
    print(splitted_calc_reaction_data)
    molecule_energies = integration(reaction, splitted_calc_reaction_data)
    print(molecule_energies)
    reaction_energy_kcal = get_energy_reaction(reaction, molecule_energies)
    
    
    return reaction_energy_kcal

tensor(nan, grad_fn=<MulBackward0>)

In [ ]:
f_svwn3(reaction['Densities'].to('cpu')[0], predictions.to('cpu')[0])

In [89]:
torch.cuda.memory_allocated()/1024**3 # GB

2.548874855041504

In [88]:
torch.cuda.empty_cache()